### Import Packages

In [ ]:
# import general modules
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
import datetime as dt

In [ ]:
# import pytorch modules
import torch
import torch.nn  as nn
import torch.nn.functional  as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms,utils
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import models
from torch.optim import lr_scheduler
import time
import copy

### Define requisite function

In [ ]:
def findcp():
    '''
    Function to find checkpoint number of saved model.
    '''
    cp = [0]
    for i in os.listdir(path+'/data/models/transfer_ad/'):
        if '_model.pth.tar' in i:
            cp.append(int(i.split('_')[0]))
    return str(max(cp)+1)

In [ ]:
def save_checkpoint(state, filename= path+'/data/models/transfer_ad/checkpoint.pth.tar'):
    '''
    Function to save checkpoint of learned model.
    '''
    torch.save(state, filename)

In [ ]:
class ImageFolderWithPaths(datasets.ImageFolder):
    '''
    Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    '''
    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

### Import data

In [ ]:
# get path
path = os.getcwd()

# define train data paths
train_dir = path + '/data/data_Aug/train/'

# define validation data paths
val_dir = path + '/data/data_Aug/val/'

# define test data path
test_dir = path + '/data/test/'

In [ ]:
# Image transformations
image_transforms = {
    't':transforms.Compose([
        transforms.Resize(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
# Datasets from folders
data = {
    'train':
    datasets.ImageFolder(root=train_dir, transform=image_transforms['t']),
    'val':
    datasets.ImageFolder(root=val_dir, transform=image_transforms['t']),
    'test':
    ImageFolderWithPaths(root=test_dir, transform=image_transforms['t'])
}

# Dataloader iterators, make sure to shuffle
dataloaders = {
    'train': DataLoader(data['train'], batch_size=128, shuffle=True, num_workers=2),
    'val': DataLoader(data['val'], batch_size=128, shuffle=True, num_workers=2),
    'test': DataLoader(data['test'], batch_size=128, num_workers=2)
}

### Define Model

In [ ]:
# define number of classses
n_class = len(data['val'].classes)

# define device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# define model
model_conv= models.inception_v3(pretrained = True)

# add final pooling layer
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Sequential(
                   nn.Linear(num_ftrs, 512),
                   nn.ReLU(inplace=True),
                   nn.Linear(512, n_class))

# multiple GPU
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_conv = nn.DataParallel(model_conv)

# send model to device
model_conv.to(device)

print('Model Defined')

In [ ]:
# create folder to save models
try:
    os.listdir(path+'/data/models/transfer_ad/')
except:
    os.mkdir(path+'/data/models/transfer_ad/')

In [ ]:
# get the labels in a dataframe
df_label = pd.DataFrame()
df_label['category'] = data['train'].classes
df_label['category'] = df_label['category'].astype(int)
df_label['c'] = df_label.index

### Training

In [ ]:
# define criterion and loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_conv.parameters())

# define number of epochs
num_epochs = 25

# define min loss 
min_loss = 10000

# train and validate
for epoch in range(num_epochs):
    
    # print the initiation of epoch
    print('{} : Epoch {}/{}'.format(dt.datetime.now(),epoch+1, num_epochs))
    print('-' * 20)
    
    
    # run training and validation phase 
    for phase in ['train', 'val']:
        
        # choose to train or eval model
        if phase == 'train':
            model_conv.train()
        else:
            model_conv.eval()

        # initiate variables
        running_loss = 0.0
        running_corrects = 0
        batch = 0
        
        # run batch training/validation
        for inputs, labels in dataloaders[phase]:
            
            # inputs
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # outputs
            if phase == 'train':
                outputs, aux = model_conv(inputs)
            else:
                outputs = model_conv(inputs)
            
            # loss
            loss = criterion(outputs, labels)

            # update gradient
            if phase == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            # get prediction
            _, preds = torch.max(outputs, 1)
            
            # append loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
            
            # print batch statistics
            batch = batch + 1
            if batch % 10 == 0:
                print('{} {} Batch:{} loss: {:.4f}, acc: {:.4f}'.format(dt.datetime.now(),
                                                                        phase,
                                                                        batch,
                                                                        running_loss/(batch * 128),
                                                                        running_corrects.double()/(batch * 128)))
        
            
        # get epoch statistics
        epoch_loss = running_loss / len(data[phase])
        epoch_acc = running_corrects.double() / len(data[phase])

        print('{} Epoch:{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                             epoch,
                                                             epoch_loss,
                                                             epoch_acc))
        
        # save best model
        if phase == 'val':
            if epoch_loss < min_loss:
                min_loss = epoch_loss
                # save model
                save_checkpoint({
                                     'epoch': epoch + 1,
                                     'arch': 'model',
                                     'state_dict': model_conv.state_dict(),
                                     'current_loss': epoch_loss,
                                     'current_acc': epoch_acc, 
                                     'optimizer' : optimizer.state_dict(),
                                 }, filename= path+'/data/models/transfer_ad/'+ findcp() + '_model.pth.tar')

### Evaluation

In [ ]:
### get Prediction
df_test = pd.DataFrame()

for inputs, labels, path in dataloaders['test']:

    # inputs
    inputs = inputs.to(device)
    
    # outputs
    outputs = model_conv(inputs)
    
    # prediction
    _, preds = torch.max(outputs, 1)
    
    temp_output = pd.DataFrame()
    temp_output['path'] = path
    temp_output['c'] = preds.to('cpu')
    df_test = df_test.append(temp_output)

# get image id from path
df_test['image_id'] = df_test['path'].apply(lambda x: int(x.split('/')[-1].split('.jpg')[0]))
del df_test['path']

# map the the labels obtained from model using df_label
df_test = df_test.merge(df_label, on = 'c', how ='left')
del df_test['c']

# sort
df_test.sort_values(by = 'image_id')

# test head
df_test.head()

# save output
df_test.to_csv(path + '/data/outputs/tl_ad_test.csv', index = False)